In [ ]:
import sys
# Add the parent directory to the system path
sys.path.insert(1, r"C:\Users\keena\Documents\University\python_scripts")

import Function_files.data_functions as dat
from Function_files.plotting_functions import plot_scope
from Function_files.filter_functions import smooth_data 
from Function_files.math_functions import average_arrays, zoom
from Function_files.fitting_functions import fit_gls, peak_find, pseudo_voigt, lorentzian, fit_lorentz

import os
import numpy as np
import matplotlib.pyplot as mp

path = r"C:\Users\keena\Desktop\Paderborn EIT"
exts = '.csv'

di = dict(

time = 0,
scan = 1,
probe = 2

)

In [ ]:
# populate folder and file lists
folder_list, file_list = dat.dir_interogate(path, extensions=exts)
# create dictionary for indexing folders and files
folder_index = dat.make_index_dict(folder_list)
file_index = [dat.make_index_dict(file_sub_list) for file_sub_list in file_list]
# lists of the data paths and metadata (text) paths
data_path_list = dat.search_paths(folder_list, file_list, exts)
metadata_path_list = dat.search_paths(folder_list, file_list, exts)
# extract the relevant data from the files
EIT_10MHz_sets = [dat.open_csv(os.path.join(path, data_path)) for data_path in data_path_list[folder_index['10MHz Scan']]]
EIT_window_sets = [dat.open_csv(os.path.join(path, data_path)) for data_path in data_path_list[folder_index['EIT window']]]
Other_sets = [dat.open_csv(os.path.join(path, data_path)) for data_path in data_path_list[folder_index['Other']]]

In [ ]:
N = []
for excel in EIT_10MHz_sets:
    a, b = zoom(excel[:,di['time']], (-0.5, 0.5))
    N.append(b-a)

# convert timebase to frequency and find N points between scans
v = 10E6 / 0.5
frequency = [excel[:,di['time']] * v for excel in EIT_10MHz_sets]
# smooth data and fidn peaks
smoothed = [smooth_data(excel[:,di['scan']]) for excel in EIT_10MHz_sets]
peaks = [peak_find(smooth, freq, top_tol=0.75, dist=n) for smooth, freq, n in zip(smoothed, frequency, N)]

In [ ]:
# isolate the individual peaks and pad around these points for fitting to a single peak
window_idx = []
for peak_set, n in zip(peaks, N):
    temp = []
    for peak in peak_set:
        start = int(peak - n/2)
        stop = int(peak + n/2)
        if start < 0:
            start = 0
        if stop > 1000000:
            stop = -1
        temp.append(np.array([start, stop]))
    window_idx.append(temp)

In [ ]:
# fit to the peaks in each plot
index = 0
l_fitted = []
vp_fitted = []
for freq, smooth, peak, indexes in zip(frequency, smoothed, peaks, window_idx):
    l_temp = []
    vp_temp = []
    for place, idx in enumerate(indexes):
        
        top = smooth[peak[place]]
        low = min(smooth[idx[0]:idx[1]])
        mid = freq[peak[place]]
        width = (freq[idx[1]]-freq[idx[0]])/2

        try:
            l_fit = fit_lorentz(freq[idx[0]:idx[1]], smooth[idx[0]:idx[1]], (top, low, mid, width))
            vp_fit = fit_gls(freq[idx[0]:idx[1]], smooth[idx[0]:idx[1]], (low, top, mid, width, top, mid, width, 0.2))
            l_temp.append(l_fit)
            vp_temp.append(vp_fit)
        except:
            l_temp.append([])
            vp_temp.append([])
    l_fitted.append(l_temp)
    vp_fitted.append(vp_temp)

In [ ]:
# plot the fitted peaks in each dataset
index = 0
for freq, smooth, peak, indexes, vp_fit in zip(frequency, smoothed, peaks, window_idx, vp_fitted):
    for idx, vp_params in zip(indexes, vp_fit):
        fig, ax = mp.subplots()
        ax.plot(freq[idx[0]:idx[1]], smooth[idx[0]:idx[1]])
        try:
            ax.plot(freq[idx[0]:idx[1]], pseudo_voigt(freq[idx[0]:idx[1]], *vp_params[0]), label='vp')
        except:
            print('fit fail')
        ax.set_title(list(file_index[folder_index['10MHz Scan']])[index])
        ax.set(xlabel='Frequency (MHz)')
        ax.legend()
    index += 1

In [ ]:
# plot the fitted peaks in each dataset
index = 0
for freq, smooth, peak, indexes, l_fit in zip(frequency, smoothed, peaks, window_idx, l_fitted):
    for idx, l_params in zip(indexes, l_fit):
        fig, ax = mp.subplots()
        ax.plot(freq[idx[0]:idx[1]], smooth[idx[0]:idx[1]], label='Data')
        ax.set_title(list(file_index[folder_index['10MHz Scan']])[index])
        ax.set(xlabel='Frequency (MHz)')
        try:
            ax.plot(freq[idx[0]:idx[1]], lorentzian(freq[idx[0]:idx[1]], *l_params[0]), label='lorentz fit')
        except:
            print('fit_fail')
        ax.legend()
    index += 1

In [ ]:
delta = []
for fitted_l in l_fitted:
    temp = []
    for fit in fitted_l:
        try:
            temp.append(fit[0][-1])
        except:
            print(fit)
    delta.append(temp)

In [ ]:
delta[8]

In [ ]:
sum(np.abs(delta[8]))/len(delta[8])

In [ ]:
probe = ['20$\mu$W', '10$\mu$W']
icon = ['o', 's']
pump = [[1.1, 2, 4, 3], [1, 2.6, 3.8, 1.5]]
nu = [[2.847351802976671, 2.1776510211506896, 3.546161795820066, 5.104464412528882],[4.3466396406829655, 1.350920823693339, 5.975702449483515, 2.8199495561250704]]

In [ ]:
fig, ax = mp.subplots()
for index, label in enumerate(probe):
    ax.plot(pump[index], nu[index], marker=icon[index], fillstyle='none', linestyle='none', label=label)
    ax.set(xlabel='Pump Power (mW)', ylabel='FHWM (MHz)')
    ax.legend(frameon=True)
